In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

#enviorement: nosql3.8

api_key="cfb0e81f9edc5ff91586f4672ba286fb"

# base_url=f"https://api.openweathermap.org/data/2.5/weather?q=Madrid&units=metric&appid={api_key}"

# response=(requests.get(base_url, params={"api_key":api_key})).json()

base_url=f"https://api.openweathermap.org/data/2.5/weather?q=Sudafrica&units=metric"

response1=(requests.get(base_url, params={"appid":api_key})).json()

df=pd.json_normalize(response1)

df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,main.sea_level,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1764083578,7200,953987,Republic of South Africa,200,26,-30,...,32,1013,852,4.2,81,2.42,2,ZA,1764040103,1764089878


City name

Temperature (°C)

Humidity (%)

Wind speed (m/s or km/h)

Weather description (e.g., “clear sky”)

In [2]:
requests=df[['name','main.temp', 'main.humidity','wind.speed', 'sys.sunrise']]
requests

,name,main.temp,main.humidity,wind.speed,sys.sunrise
0,Republic of South Africa,24.06,32,4.2,1764040103


In [116]:
paises_por_continente = [
    "Egipto",
    "Nigeria",
    "Sudafrica",
    "Canadá",
    "Brasil",
    "México",
    "Argentina",
    "Japon",
    "India",
    "China",
    "Tailandia",
    "Francia",
    "Alemania",
    "España",
    "Italia",
    "Australia"
]

result=[]

for i in paises_por_continente:

    base_urli=f"https://api.openweathermap.org/data/2.5/weather?q={i}&units=metric"

    response=(requests.get(base_urli, params={"appid":api_key})).json()

    df_fila=pd.json_normalize(response)

    df_fila=df_fila[['name','main.temp', 'main.humidity','wind.speed', 'sys.sunrise']]

    result.append(df_fila)

df_final=pd.concat(result, ignore_index=True)
df_final



    

,name,main.temp,main.humidity,wind.speed,sys.sunrise
0,Egypt,16.93,39,4.01,1764044860
1,Nigeria,27.20,22,2.55,1764048416
2,Republic of South Africa,22.01,37,4.29,1764040103
3,Canadá,33.61,37,2.01,1764057981
4,Brazil,25.11,80,3.03,1764061696
5,Mexico City,19.97,37,0.00,1764075051
6,Argentina,36.33,16,6.59,1764061163
7,Japan,9.92,84,1.54,1764106050
8,Innichen,-2.00,100,1.54,1764052051
9,China,27.64,62,6.04,1764075745


In [ ]:
df_final.to_csv('PaisesTempetaura.csv')

4. Data Analysis

Answer questions such as:
Which cities are the hottest and coldest right now?


In [117]:
max=df_final["name"][df_final["main.temp"] == df_final["main.temp"].max()] ## hay que hacer un .item() para coger solo para un elemento
min=df_final["name"][df_final["main.temp"] == df_final["main.temp"].min()]


In [118]:
print(f'El pais mas caliente que es: {max.item()} y el pais mas frio es: {min.item()}')

El pais mas caliente que es: Argentina y el pais mas frio es: Innichen


Is there a relationship between temperature and humidity?

In [120]:
df_final.columns

Index(['name', 'main.temp', 'main.humidity', 'wind.speed', 'sys.sunrise'], dtype='object')

In [123]:
## Acordarse si se usa f "" al usar los valores dentro se usa ' '
print(f"La relacion que hay que hay entre temperatura y humedad es {df_final['main.temp'].corr(df_final['main.humidity']).round(4)}")

La relacion que hay que hay entre temperatura y humedad es -0.7053


Which continent seems to have the windiest cities?